In [1]:
import psycopg2 as pg2
import pandas as pd
import time
import numpy as np
import sys
import datetime
import pickle

from src.helpers import *
from src.model import Model

/Users/raficg/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [14]:
data = pd.read_csv('data/test/dates.csv')

In [15]:
X, y = clean_data(data)

In [16]:
X.shape

(10238, 20)

In [ ]:
query = 'CREATE TABLE reservations (\n'
for col, t in zip(data.dtypes.index, data.dtypes.values):
    query += col + ' '
    if t == int:
        query += 'INTEGER,\n'
    elif t == float:
        query += 'DECIMAL(5,2),\n'
    else:
        query += 'VARCHAR(50),\n'
query += ');'

In [4]:
query = 'CREATE TABLE reservations (\nid INTEGER,\nhotel VARCHAR(50),\ntimestamp INTEGER,\nis_canceled INTEGER,\nlead_time INTEGER,\narrival_date_year INTEGER,\narrival_date_month VARCHAR(50),\narrival_date_week_number INTEGER,\narrival_date_day_of_month INTEGER,\nstays_in_weekend_nights INTEGER,\nstays_in_week_nights INTEGER,\nadults INTEGER,\nchildren DECIMAL(5,2),\nbabies INTEGER,\nmeal VARCHAR(50),\ncountry VARCHAR(50),\nmarket_segment VARCHAR(50),\ndistribution_channel VARCHAR(50),\nis_repeated_guest INTEGER,\nprevious_cancellations INTEGER,\nprevious_bookings_not_canceled INTEGER,\nreserved_room_type VARCHAR(50),\nassigned_room_type VARCHAR(50),\nbooking_changes INTEGER,\ndeposit_type VARCHAR(50),\nagent DECIMAL(5,2),\ncompany DECIMAL(5,2),\ndays_in_waiting_list INTEGER,\ncustomer_type VARCHAR(50),\nadr DECIMAL(5,2),\nrequired_car_parking_spaces INTEGER,\ntotal_of_special_requests INTEGER,\nreservation_status VARCHAR(50),\ntotal_nights INTEGER,\nparty_size DECIMAL(5,2),\nis_family INTEGER,\nroom_difference INTEGER,\nhad_agent INTEGER,\nhad_company INTEGER);'

In [ ]:
#COPY reservations(id, hotel, timestamp, is_canceled, lead_time, arrival_date_year, arrival_date_month, arrival_date_week_number, arrival_date_day_of_month, stays_in_weekend_nights, stays_in_week_nights, adults, children, babies, meal, country, market_segment, distribution_channel, is_repeated_guest, previous_cancellations, previous_bookings_not_canceled, reserved_room_type, assigned_room_type, booking_changes, deposit_type, agent, company, days_in_waiting_list, customer_type, adr, required_car_parking_spaces, total_of_special_requests, reservation_status, total_nights, party_size, is_family, room_difference, had_agent, had_company)
#FROM '/home/data/Galvanize/capstones/capstone-3/hotel-cancellations/data/test/dates.csv'
#DELIMITER ','
#CSV HEADER;

In [2]:
conn = pg2.connect(dbname='hotels', user='postgres', 
                   host='localhost', port='5432', 
                   password='password')

In [3]:
cur = conn.cursor()

In [4]:
conn.autocommit = True

In [16]:
query = '''
    SELECT
    id, timestamp,
    arrival_date_year, arrival_date_month, arrival_date_day_of_month,
    gross_profit, cancel_prob, hotel, predicted_demand, total_nights, adr
    FROM predictions
    ORDER BY timestamp DESC
    '''
cur.execute(query)
obj = cur.fetchall()
df = pd.DataFrame(obj, columns=['id', 'timestamp',
                          'arrival_date_year',
                          'arrival_date_month',
                          'arrival_date_day_of_month',
                          'gross_profit',
                          'cancel_prob',
                          'hotel',
                          'predicted_demand', 
                          'total_nights', 
                          'adr'])

In [30]:
profits = []
dates = []
for day in np.arange(1, 32):
    adr = round(df[(df['arrival_date_month'] == 'August') & 
               (day >= df['arrival_date_day_of_month']) & 
               (day < (df['arrival_date_day_of_month'] + df['total_nights']))]['adr'].mean(), 2)
    rooms_occ = int(df[(df['arrival_date_month'] == 'August') & 
               (day >= df['arrival_date_day_of_month']) & 
               (day < (df['arrival_date_day_of_month'] + df['total_nights']))].groupby('hotel')['predicted_demand'].mean().sum())
    
    profits.append(adr * rooms_occ)
    
    dates.append('8/' + str(day))

In [31]:
profits

[102872.95000000001,
 103122.0,
 105955.68,
 110839.1,
 116193.45,
 118888.6,
 123046.0,
 121580.40000000001,
 123841.98000000001,
 121929.39,
 122543.84999999999,
 124153.04000000001,
 122360.04,
 125690.16,
 125155.68,
 125182.35,
 125820.64,
 126933.93999999999,
 127081.09999999999,
 125802.96,
 122214.84,
 119922.16,
 116441.0,
 115858.4,
 116669.2,
 115459.20000000001,
 113888.67,
 114326.28,
 110139.84,
 105865.28,
 99668.36]

In [19]:
round(df[(df['arrival_date_month'] == 'August') & 
   (3 >= df['arrival_date_day_of_month']) & 
   (3 < (df['arrival_date_day_of_month'] + df['total_nights']))]['adr'].mean(), 2)

165.04

In [20]:
df = pd.DataFrame(obj, columns=['id', 'timestamp', 'hotel', 'party_size', 'market_segment',
                                'room_difference', 'booking_changes', 'total_of_special_requests', 'total_nights', 
                                'arrival_year', 'arrival_month', 'arrival_day'])

In [36]:
dates = []
for y, m, d in zip(df['arrival_year'], df['arrival_month'], df['arrival_day']):
    date = str(y) + '-' + str(m) + '-' + str(d)
    dates.append(datetime.datetime.strptime(date, '%Y-%B-%d'))
dates = pd.DatetimeIndex(dates)

In [10]:
X, _ = clean_data(df, y_included=False)

In [12]:
X.shape

(10238, 20)

In [8]:
# query = '''CREATE TABLE predictions (
#             id INTEGER,
#             hotel VARCHAR(50),
#             timestamp INTEGER,
#             arrival_date_year INTEGER,
#             arrival_date_month VARCHAR(50),
#             arrival_date_day_of_month INTEGER,
#             adr DECIMAL(5,2),
#             total_nights INTEGER,
#             gross_profit DECIMAL(6,2),
#             cancel_prob DECIMAL(4,3),
#             predicted_demand INTEGER);
#         '''
# cur.execute(query)

In [21]:
with open('src/model.pkl', 'rb') as f:
        model = pickle.load(f)
model = pickle.load(f)

ModuleNotFoundError: No module named 'model'

In [26]:
query = '''
        SELECT
        id, timestamp, arrival_date_year,
        arrival_date_month, arrival_date_day_of_month,
        hotel, party_size, market_segment, room_difference, 
        booking_changes, total_of_special_requests, 
        adr, total_nights, (adr * total_nights) as gross_profit
        FROM reservations r
        WHERE r.id NOT IN (SELECT id from predictions)
        '''
cur.execute(query)
obj = cur.fetchall()

In [27]:
df = pd.DataFrame(obj, columns=['id', 'timestamp',  
                                'arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month',
                                'hotel', 'party_size', 'market_segment', 'room_difference',
                                'booking_changes', 'total_of_special_requests',
                                'adr', 'total_nights', 'gross_profit'])

In [16]:
X, _ = clean_data(df, y_included=False)

In [ ]:
dates = []
    for y, m, d in zip(df['arrival_year'], df['arrival_month'], df['arrival_day']):
        date = str(y) + '-' + str(m) + '-' + str(d)
        dates.append(datetime.datetime.strptime(date, '%Y-%B-%d'))
    dates = pd.DatetimeIndex(dates)
    probs, bookings = model.predict_proba(X, dates)

In [ ]:
df['cancel_prob'] = probs
df['predicted_demand'] = bookings

In [ ]:
id INTEGER,
#             hotel VARCHAR(50),
#             timestamp INTEGER,
#             arrival_date_year INTEGER,
#             arrival_date_month VARCHAR(50),
#             arrival_date_day_of_month INTEGER,
#             adr DECIMAL(5,2),
#             total_nights INTEGER,
#             gross_profit DECIMAL(6,2),
#             cancel_prob DECIMAL(4,3),
#             predicted_demand INTEGER);

In [ ]:
query = 'INSERT INTO predictions\nVALUES '
for row, item in df.iterrows():
    query += ('(%d, %s, %d, %d, %s, %d, %5.2f, %d, %6.2f, %4.3f, %d)' % 
              (item['id'], item['hotel'], item['timestamp'],
               item['arrival_date_year'], item['arrival_date_month'],
               item['arrival_date_day_of_month'], item['adr'], 
               item['total_nights'], item['gross_profit'], 
               item['cancel_prob'], item['predicted_demand'])
    query += '\n'
query += ';'
cur.execute(query)
